# Data Augmentation

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 4 classes.
Found 54 images belonging to 4 classes.


# NASNetMobile

In [30]:
from keras.applications.nasnet import NASNetMobile

NASNet = NASNetMobile(input_shape=(224,224,3), include_top=False, weights='imagenet')

In [39]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, models

# Freeze all layers
for layer in NASNet.layers:
    layer.trainable = False
# Unfreeze last 15 layers
for layer in NASNet.layers[-15:]:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in NASNet.layers:
    if layer.trainable == True:
        print(layer, layer.trainable)

model = models.Sequential()
model.add(NASNet)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

<keras.layers.normalization.BatchNormalization object at 0xb4558de10> True
<keras.layers.normalization.BatchNormalization object at 0xb4549eef0> True
<keras.layers.normalization.BatchNormalization object at 0xb45400e10> True
<keras.layers.normalization.BatchNormalization object at 0xb461a62b0> True
<keras.layers.pooling.AveragePooling2D object at 0xb46113198> True
<keras.layers.pooling.AveragePooling2D object at 0xb4560cb70> True
<keras.layers.pooling.AveragePooling2D object at 0xb460d3668> True
<keras.layers.normalization.BatchNormalization object at 0xb45fa19e8> True
<keras.layers.merge.Add object at 0xb4560c470> True
<keras.layers.merge.Add object at 0xb4545df98> True
<keras.layers.merge.Add object at 0xb4560c940> True
<keras.layers.merge.Add object at 0xb46096d68> True
<keras.layers.merge.Add object at 0xb4602ab00> True
<keras.layers.merge.Concatenate object at 0xb45639400> True
<keras.layers.core.Activation object at 0xb49256780> True


In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 7, 7, 1056)        4269716   
_________________________________________________________________
flatten_10 (Flatten)         (None, 51744)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               13246720  
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 771       
Total params: 17,517,207
Trainable params: 13,249,251
Non-trainable params: 4,267,956
_________________________________________________________________


In [ ]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )
model.save('NASNet_1.h5')